In [8]:
#Celda para librerías
import sklearn as sk


import matplotlib as plt
import matplotlib.pyplot as plt

import pandas as pd
import seaborn as sns
import numpy as np
import plotly.graph_objects as go

from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import RobustScaler
from sklearn.neighbors import NearestNeighbors

# LightGBM
from lightgbm import LGBMClassifier
import lightgbm as lgb

# XGBoost
from xgboost import XGBClassifier
#KNN
from sklearn.neighbors import KNeighborsClassifier
#Neural Network
from sklearn.neural_network import MLPClassifier
#Random Forest
from sklearn.ensemble import RandomForestClassifier


from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score


import json

import warnings
warnings.filterwarnings('ignore')

print(np.__version__)
from ydata_profiling import ProfileReport
import os
print("Directorio actual:", os.getcwd())



# Have all columns appear when dataframes are displayed.
pd.set_option('display.max_columns', None) 
# Have 100 rows appear when a dataframe is displayed
pd.set_option('display.max_rows', 500)
# Display dimensions whenever a dataframe is printed out.
pd.set_option('display.show_dimensions', True)

#Importando los datos

app_train_def = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/application_train_preprocesado_definitivo_v1.csv')
print(app_train_def.info())

1.23.5
Directorio actual: c:\Users\Yeray\Desktop\DATA_SCIENCE_ML\Home-Credit-TFG\JUPYTER_NOTEBOOKS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 307 entries, TARGET to WALLSMATERIAL_MODE_Wooden
dtypes: bool(127), float64(138), int64(42)
memory usage: 459.5 MB
None


In [9]:
#Train y Test
X = app_train_def.drop(columns = ['TARGET'])
y = app_train_def['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train.columns = X_train.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)
X_test.columns = X_test.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)

In [10]:
#Definimos espacio de hiperparámetros para LightGBM
space_lightGBM = {
    'boosting_type': 'gbdt',  # Fijo
    'objective': 'binary',   # Fijo
    'metric': 'auc',         # Fijo

    'num_leaves': hp.quniform('num_leaves', 20, 40, 1), 
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.03),  
    'min_child_samples': hp.quniform('min_child_samples', 50, 90, 1),  
    'min_child_weight': hp.uniform('min_child_weight', 0.001, 0.02),
    'min_gain_to_split': hp.uniform('min_gain_to_split', 0.001, 0.1),
    'max_bin': hp.quniform('max_bin', 250, 350, 10),
    'feature_fraction': hp.uniform('feature_fraction', 0.2, 0.5),
    'max_depth': hp.choice('max_depth', [-1, 3, 5]), 
    'bagging_fraction': hp.uniform('bagging_fraction', 0.5, 1.0),
    'bagging_freq': hp.quniform('bagging_freq', 1, 5, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0, 110),  
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 0.2),  
    
    'scale_pos_weight': 1,  # Fijo
    'is_unbalance': False  # Fijo
}

Trials_lightGBM_Ligero = Trials()


In [ ]:
n_iter = 200

def F_OPT(params):
    params['max_bin'] = int(params['max_bin'])
    params['num_leaves'] = int(params['num_leaves'])
    params['min_child_samples'] = int(params['min_child_samples'])
    params['bagging_freq'] = int(params['bagging_freq'])
    params['min_child_samples'] = int(params['min_child_samples'])

    model = LGBMClassifier(
        n_estimators=n_iter,
        boosting_type=params['boosting_type'],
        objective=params['objective'],
        metric=params['metric'],
        num_leaves=params['num_leaves'],
        learning_rate=params['learning_rate'],
        min_child_samples=params['min_child_samples'],
        min_child_weight=params['min_child_weight'],
        min_gain_to_split=params['min_gain_to_split'],
        max_bin=params['max_bin'],
        feature_fraction=params['feature_fraction'],
        max_depth=params['max_depth'],
        bagging_fraction=params['bagging_fraction'],
        bagging_freq=params['bagging_freq'],
        reg_lambda=params['reg_lambda'],
        reg_alpha=params['reg_alpha'],
        scale_pos_weight=params['scale_pos_weight'],
        is_unbalance=params['is_unbalance'],
        random_state=42)
    
    auc_lightGBM_Ligero = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc').mean()
    return {'loss': -auc_lightGBM_Ligero, 'status': STATUS_OK}
    

In [14]:
best_lightGBM_Ligero = fmin(fn=F_OPT,
                            space=space_lightGBM,
                            algo=tpe.suggest,
                            max_evals=15,
                            trials=Trials_lightGBM_Ligero,
                            rstate= np.random.default_rng(42)
)

best_lightGBM_Ligero['num_leaves'] = int(best_lightGBM_Ligero['num_leaves'])
best_lightGBM_Ligero['max_bin'] = int(best_lightGBM_Ligero['max_bin'])
best_lightGBM_Ligero['min_child_samples'] = int(best_lightGBM_Ligero['min_child_samples'])
best_lightGBM_Ligero['bagging_freq'] = int(best_lightGBM_Ligero['bagging_freq'])
best_lightGBM_Ligero['min_child_samples'] = int(best_lightGBM_Ligero['min_child_samples'])

print(best_lightGBM_Ligero)

  0%|          | 0/15 [00:00<?, ?trial/s, best loss=?]

  File "C:\Users\Yeray\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")



[LightGBM] [Warning] feature_fraction is set=0.4969651399430127, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4969651399430127
[LightGBM] [Warning] min_gain_to_split is set=0.0655224975751405, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0655224975751405
[LightGBM] [Warning] bagging_fraction is set=0.5304174185556783, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5304174185556783
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.4969651399430127, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4969651399430127
[LightGBM] [Warning] min_gain_to_split is set=0.0655224975751405, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0655224975751405
[LightGBM] [Warning] bagging_fraction is set=0.5304174185556783, subsample=1.0 will be ignored. Current value: bagging_fraction=0

In [15]:
import pickle
with open("Trials_LightGBM_Ligero.pkl", "wb") as f:
    pickle.dump(Trials_lightGBM_Ligero, f)
with open("Trials_lightGBM_Ligero.pkl", "rb") as f:
    Trials_lightGBM_Ligero = pickle.load(f)

In [17]:
#Mostramos los resultados de los trials 
from tabulate import tabulate

def trials_to_dataframe(trials):
    """
    Convierte la info de 'trials' de Hyperopt en un DataFrame 
    con columnas relevantes (loss, hiperparámetros, etc.).
    """
    rows = []
    for trial in trials.trials:
        # trial['result']['loss'] -> la métrica
        # trial['misc']['vals'] -> diccionario de hiperparámetros propuestos
        loss = trial['result']['loss']
        vals = trial['misc']['vals']
        
        # Convertir vals a algo "plano"
        row = {**vals}
        row['loss'] = loss
        rows.append(row)
    
    # df con columnas = keys (vals + 'loss')
    df = pd.DataFrame(rows)
    
    # A veces los hps que eran hp.quniform se quedan en listas => df['num_leaves'].apply(lambda x: x[0] if isinstance(x, list) else x)
    for c in df.columns:
        df[c] = df[c].apply(lambda x: x[0] if isinstance(x, list) else x)
    
    return df

# Para usarlo:
results_df = trials_to_dataframe(Trials_lightGBM_Ligero)
# Ordenamos por 'loss' asc => AUC mayor es 'loss' menor
results_df_sorted = results_df.sort_values(by='loss', ascending=True)
best_10 = results_df_sorted.head(10)
print(tabulate(best_10, headers='keys', tablefmt='psql'))

+----+--------------------+----------------+--------------------+-----------------+-----------+-------------+---------------------+--------------------+---------------------+--------------+-------------+--------------+-----------+
|    |   bagging_fraction |   bagging_freq |   feature_fraction |   learning_rate |   max_bin |   max_depth |   min_child_samples |   min_child_weight |   min_gain_to_split |   num_leaves |   reg_alpha |   reg_lambda |      loss |
|----+--------------------+----------------+--------------------+-----------------+-----------+-------------+---------------------+--------------------+---------------------+--------------+-------------+--------------+-----------|
|  7 |           0.506587 |              1 |           0.416329 |       0.0282566 |       280 |           2 |                  80 |         0.00365639 |          0.034361   |           35 |  0.166018   |      7.07155 | -0.758218 |
| 12 |           0.698999 |              3 |           0.368491 |       0.02